In [3]:
# Import modules
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [4]:
# define and open url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

soup = BeautifulSoup(page, "lxml")


In [5]:
# locate table with postal codes
table = soup.find('table', class_='wikitable sortable')

# create empty lists
postalcode = []
borough = []
neighborhood = []

# iterate through table and for each row append into respective lists
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells) == 3:
        postalcode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighborhood.append(cells[2].find(text=True))

# create a new dataframe
column_names = ['Postal Code','Borough','Neighborhood']
df = pd.DataFrame(columns=column_names)

# populate the dataframe
df['Postal Code'] = postalcode
df['Borough'] = borough
df['Neighborhood'] = neighborhood

# replace "\n" in dataframe
df.replace('\n','',regex=True, inplace=True)

# drop rows with borough that is Not Assigned
df.drop(df[df['Borough']=="Not assigned"].index,inplace=True)
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


df.shape

In [6]:
# pip install geocoder

import geocoder # import geocoder

In [7]:
# create function to retrieve latitude and longitude of a postal code

def latlong(postalcode):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postalcode))
      lat_lng_coords = g.latlng

    latitude = str(lat_lng_coords[0])
    longitude = lat_lng_coords[1]
    
    # put results in a list
    latlong = [str(latitude), str(longitude)]
    
    return latlong



In [8]:
# get list of unique postal codes from dataframe
postallist = {}
postallist = df['Postal Code'].unique()

# create a new dataframe
postal_columns = ['Postal Code', 'latlong', 'Latitude', 'Longitude']
postal_frame = pd.DataFrame(columns=postal_columns)

# append postal code into new dataframe and retrieve latitude and longitude with function
postal_frame['Postal Code'] = postallist
postal_frame['latlong'] = postal_frame['Postal Code'].apply(latlong)

# split the column into individual latitude and longitude columns
postal_frame[['Latitude','Longitude']] = pd.DataFrame(postal_frame.latlong.tolist(),index=postal_frame.index)
postal_frame.drop(columns='latlong')
postal_frame


,Postal Code,latlong,Latitude,Longitude
0,M3A,"[43.75293455500008, -79.33564142299997]",43.75293455500008,-79.33564142299997
1,M4A,"[43.72810248500008, -79.31188987099995]",43.72810248500008,-79.31188987099995
2,M5A,"[43.65096410900003, -79.35304116399999]",43.65096410900003,-79.35304116399999
3,M6A,"[43.723265465000054, -79.45121077799996]",43.723265465000054,-79.45121077799996
4,M7A,"[43.66179000000005, -79.38938999999993]",43.66179000000005,-79.38938999999993
...,...,...,...,...
98,M8X,"[43.65333956200004, -79.50976584199998]",43.65333956200004,-79.50976584199998
99,M4Y,"[43.66665858500005, -79.38147173199997]",43.66665858500005,-79.38147173199997
100,M7Y,"[43.648700000000076, -79.38544999999993]",43.648700000000076,-79.38544999999993
101,M8Y,"[43.632798483000045, -79.49301657299998]",43.632798483000045,-79.49301657299998


In [9]:
# merge latitude and longitude into dataframe with borough and neighborhood
result = pd.merge(df,postal_frame,left_on='Postal Code',right_on='Postal Code',how='left')

# drop latlong column as it is not needed
result = result.drop(columns='latlong')
toronto_data = result[result['Borough'].str.contains('Toronto')]
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65096410900003,-79.35304116399999
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66179000000005,-79.38938999999993
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65749059800004,-79.37752923699998
15,M5C,Downtown Toronto,St. James Town,43.65173364700007,-79.37555358799995
19,M4E,East Toronto,The Beaches,43.67814827600006,-79.29534930999995
20,M5E,Downtown Toronto,Berczy Park,43.645195888000046,-79.37385548899994
24,M5G,Downtown Toronto,Central Bay Street,43.65607218800005,-79.38565318999997
25,M6G,Downtown Toronto,Christie,43.668602116000045,-79.42038731299994
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.65054164500003,-79.38411611599997
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.66491031700008,-79.43866366999998


In [10]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood, code in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], 
                                                 toronto_data['Neighborhood'], toronto_data['Postal Code']):
    label = '{},{}, {}'.format(code, neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
# define foursquare credentials
CLIENT_ID = 'XALZ5A2NK4K1B2F3PLDUMTFTTL4DSELCLC5ZA5EBCEX5I511' # your Foursquare ID
CLIENT_SECRET = 'NAWFRPYQOCKJBPW50RTJAO0IPNG1QOMLINDCFF2UJBN0DVRT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XALZ5A2NK4K1B2F3PLDUMTFTTL4DSELCLC5ZA5EBCEX5I511
CLIENT_SECRET:NAWFRPYQOCKJBPW50RTJAO0IPNG1QOMLINDCFF2UJBN0DVRT


In [16]:
# function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        nb_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in nb_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# get nearby venues for toronto
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.015152,0.00000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00,0.00,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.02439,0.000000,0.000000,0.024390,0.000000,0.024390,0.000000,0.0,...,0.02439,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.00000,0.000000,0.020000,0.010000,0.000000,0.000000,0.020000,0.0,...,0.00000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.0,...,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.00000,0.000000,0.000000,0.000000,0.018182,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00,0.00,0.000000,0.018182,0.018182,0.018182,0.000000,0.000000
5,Christie,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.011236,0.00000,0.011236,0.011236,0.000000,0.000000,0.011236,0.000000,0.0,...,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.011236,0.000000
7,"Commerce Court, Victoria Hotel",0.010000,0.00000,0.000000,0.040000,0.010000,0.000000,0.000000,0.010000,0.0,...,0.00000,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.038462,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.00000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
# show top 5 venues for each neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2      Beer Bar  0.03
3         Hotel  0.03
4           Pub  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.07
1             Coffee Shop  0.07
2  Thrift / Vintage Store  0.05
3               Gift Shop  0.05
4               Pet Store  0.02


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2           Restaurant  0.04
3  Japanese Restaurant  0.03
4      Thai Restaurant  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
               venue  freq
0        Coffee Shop  0.06
1               Café  0.06
2  French Restaurant  0.05
3               Park  0.05
4             Lounge  0.03


----Central Bay Street----
                 venue  

In [21]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Pub,Bakery,Beer Bar,Lounge,Seafood Restaurant,Cheese Shop,Breakfast Spot,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Gift Shop,Dance Studio,Japanese Restaurant,Restaurant,Supermarket,Indian Restaurant,Pizza Place
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Japanese Restaurant,Sushi Restaurant,Tea Room,Sandwich Place,Burrito Place,Seafood Restaurant,Bookstore
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Park,French Restaurant,Bar,Speakeasy,Lounge,Restaurant,Gym / Fitness Center,Italian Restaurant
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Clothing Store,Café,Sandwich Place,Plaza,Bubble Tea Shop,Middle Eastern Restaurant,Spa,Department Store


In [23]:
## Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 0, 0, 0, 2])

In [27]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65096410900003,-79.35304116399999,0,Pub,Coffee Shop,Café,Athletics & Sports,Bank,Mexican Restaurant,Seafood Restaurant,Cosmetics Shop,Food Truck,French Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66179000000005,-79.38938999999993,0,Coffee Shop,Café,Sushi Restaurant,Hobby Shop,Distribution Center,Sandwich Place,Restaurant,Pharmacy,Park,Middle Eastern Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65749059800004,-79.37752923699998,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Cosmetics Shop,Italian Restaurant,Bar,Hotel,Café,Bakery
15,M5C,Downtown Toronto,St. James Town,43.65173364700007,-79.37555358799995,0,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Lingerie Store,Clothing Store,Theater
19,M4E,East Toronto,The Beaches,43.67814827600006,-79.29534930999995,0,Health Food Store,Pub,Trail,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [29]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",0,Pub,Coffee Shop,Café,Athletics & Sports,Bank,Mexican Restaurant,Seafood Restaurant,Cosmetics Shop,Food Truck,French Restaurant
4,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Café,Sushi Restaurant,Hobby Shop,Distribution Center,Sandwich Place,Restaurant,Pharmacy,Park,Middle Eastern Restaurant
9,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Cosmetics Shop,Italian Restaurant,Bar,Hotel,Café,Bakery
15,St. James Town,0,Coffee Shop,Café,Cocktail Bar,Seafood Restaurant,Gastropub,American Restaurant,Cosmetics Shop,Lingerie Store,Clothing Store,Theater
19,The Beaches,0,Health Food Store,Pub,Trail,Women's Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
20,Berczy Park,0,Coffee Shop,Cocktail Bar,Pub,Bakery,Beer Bar,Lounge,Seafood Restaurant,Cheese Shop,Breakfast Spot,Restaurant
24,Central Bay Street,0,Coffee Shop,Japanese Restaurant,Clothing Store,Café,Sandwich Place,Plaza,Bubble Tea Shop,Middle Eastern Restaurant,Spa,Department Store
30,"Richmond, Adelaide, King",0,Coffee Shop,Café,Clothing Store,Restaurant,Salad Place,Gym,Hotel,Sushi Restaurant,Thai Restaurant,Deli / Bodega
37,"Little Portugal, Trinity",0,Bar,Coffee Shop,Cocktail Bar,Asian Restaurant,Restaurant,Wine Bar,Yoga Studio,Japanese Restaurant,Brewery,Juice Bar
42,"Toronto Dominion Centre, Design Exchange",0,Coffee Shop,Hotel,Café,Restaurant,Deli / Bodega,Japanese Restaurant,Italian Restaurant,American Restaurant,Salad Place,Seafood Restaurant


In [36]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,"Forest Hill North & West, Forest Hill Road Park",1,Gym / Fitness Center,Astrologer,Women's Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [37]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Christie,2,Café,Grocery Store,Playground,Candy Store,Park,Coffee Shop,Baby Store,Elementary School,Eastern European Restaurant,Electronics Store
31,"Dufferin, Dovercourt Village",2,Park,Pet Store,Café,Smoke Shop,Bank,Bakery,Pool,Athletics & Sports,Furniture / Home Store,Grocery Store
36,"Harbourfront East, Union Station, Toronto Islands",2,Fast Food Restaurant,Harbor / Marina,Farm,Park,Theme Park,Women's Store,Dog Run,Fish & Chips Shop,Farmers Market,Falafel Restaurant
41,"The Danforth West, Riverdale",2,Bus Line,Business Service,Park,Grocery Store,Discount Store,Women's Store,Donut Shop,Fast Food Restaurant,Farmers Market,Farm
61,Lawrence Park,2,Bus Line,Swim School,Women's Store,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant
67,Davisville North,2,Food & Drink Shop,Breakfast Spot,Bus Line,Gym / Fitness Center,Park,Department Store,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
69,"High Park, The Junction South",2,Park,Sandwich Place,Gym / Fitness Center,Residential Building (Apartment / Condo),Gas Station,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant
73,"North Toronto West, Lawrence Park",2,Playground,Gym Pool,Park,Garden,Women's Store,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
83,"Moore Park, Summerhill East",2,Convenience Store,Gym,Park,Restaurant,Distribution Center,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant
91,Rosedale,2,Playground,Park,Grocery Store,Candy Store,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [38]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Studio District,3,Business Service,Government Building,Night Market,Baseball Field,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [39]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Roselawn,4,Spa,Home Service,Dog Run,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Ethiopian Restaurant
